In [1]:
import torch

torch.manual_seed(0)

In [2]:
X = torch.randn(10, 4, requires_grad = True)
W = torch.randn(4, 4, requires_grad = True)
Y = torch.randn(10, 4, requires_grad = True)
print("X = ", X)
print("W = ", W)
print("Y = ", Y)

X =  tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530],
        [-0.2159, -0.7425,  0.5627,  0.2596],
        [-0.1740, -0.6787,  0.9383,  0.4889],
        [ 1.2032,  0.0845, -1.2001, -0.0048],
        [-0.5181, -0.3067, -1.5810,  1.7066]], requires_grad=True)
W =  tensor([[ 0.2055, -0.4503, -0.5731, -0.5554],
        [ 0.5943,  1.5419,  0.5073, -0.5910],
        [-1.3253,  0.1886, -0.0691, -0.4949],
        [-1.4959, -0.1938,  0.4455,  1.3253]], requires_grad=True)
Y =  tensor([[ 1.5091,  2.0820,  1.7067,  2.3804],
        [-1.1256, -0.3170, -1.0925, -0.0852],
        [ 0.3276, -0.7607, -1.5991,  0.0185],
        [-0.7504,  0.1854,  0.6211,  0.6382],
        [-0.0033, -0.5344,  1.1687,  0.3945],
        [ 1.9415,  0.7915, -0.0203, -0.4372],
        [-0.2188, -2.43

In [3]:
A = torch.max(torch.mm(X, W), torch.zeros_like(X)) - Y
f = torch.trace(torch.mm(A.t(), A))
print("f = ", f)

f =  tensor(99.9048, grad_fn=<TraceBackward>)


函数 $f=||max(XW, 0)-Y||^2_F$ 求导： 

In [4]:
f.backward()
print("W = ", W.grad)
print("X = ", X.grad)
print("Y = ", Y.grad)

W =  tensor([[ 18.2980,   2.7573,   2.3914,  -0.1974],
        [ 11.0817,   6.6428,   2.5163, -20.3225],
        [ -8.6662,   3.4506,  -1.8979,  -3.3608],
        [-21.1681,  -6.6739,  -1.0693,  27.0278]])
X =  tensor([[  1.1002,   0.0860,   5.3377,   0.2788],
        [  0.9583,  10.4633, -13.5234, -16.3639],
        [ -0.8712,  -0.9272,  -0.7764,   2.0790],
        [ -1.4504,   5.6914,   0.7613,  -0.9693],
        [ -1.2892,  -3.4714,  -1.9788,   4.8091],
        [ -4.0523,  -4.3127,  -3.6114,   9.6703],
        [ -0.7312,  -0.7782,  -0.6516,   1.7449],
        [ -0.8191,  -0.8718,  -0.7300,   1.9547],
        [  1.0350,   2.9930,  -6.6743,  -7.5333],
        [ -2.4616,  -2.4243,  -2.1164,   5.7128]])
Y =  tensor([[ 2.8885e+00,  4.1639e+00,  3.4134e+00,  3.0501e+00],
        [-1.0589e+01, -2.7045e+00, -2.1849e+00, -1.7039e-01],
        [ 6.5523e-01, -1.5214e+00, -3.1982e+00, -1.5687e+00],
        [-1.5009e+00, -3.8551e+00,  4.9843e-01,  1.2764e+00],
        [-6.6077e-03, -1.0689e+00, 

首先求出 $\max'(XW,0)$

In [5]:
max_grad = torch.max(X.mm(W), torch.zeros_like(X.mm(W))) > 0
max_grad

tensor([[ True, False, False,  True],
        [ True,  True, False, False],
        [False, False, False,  True],
        [False,  True,  True, False],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [ True, False, False, False],
        [False, False,  True,  True]])

验证
$\frac{\partial f}{\partial W}
= 2 X^T[(\max(XW,0)-Y) \odot \max'(XW,0)]
$

In [6]:
W.grad == 2 * torch.mm(X.t(), (torch.max(X.mm(W), torch.zeros_like(X.mm(W))) - Y) * max_grad)

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

验证
$\frac{\partial f}{\partial X}
= 2(\max(XW,0)-Y) \odot \max'(XW,0) W^T
$

In [7]:
X.grad == 2* ((torch.max(X.mm(W), torch.zeros_like(X.mm(W))) - Y) * max_grad).mm(W.t())

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

验证
$\frac{\partial f}{\partial Y}
= 2(Y-\max(XW,0))
$

In [8]:
Y.grad == 2 * (Y - torch.max(torch.mm(X, W), torch.zeros_like(X.mm(W))))

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])